<a href="https://colab.research.google.com/github/UDICatNCHU/SparkTutorial/blob/answer/(Spark%20Tutorial)%20%E4%B8%8A%E8%AA%B2%E8%AC%9B%E7%BE%A9%20(%E8%A7%A3%E7%AD%94)%202019%20Spark%20on%20PM25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 環境初始化 (大約三至五分鐘)
! wget -O init_env.sh https://www.dropbox.com/s/6bnwn8u2hz19s59/init_env.sh && \
bash init_env.sh

In [ ]:
import os, sys
os.environ['SPARK_HOME'] = "/usr/local/spark"
os.environ['PYSPARK_PYTHON'] = "/usr/local/bin/python"
sys.path.append("/usr/local/spark/python/")
sys.path.append("/usr/local/spark/python/lib/pyspark.zip")
sys.path.append("/usr/local/spark/python/lib/py4j-0.10.4-src.zip")

In [ ]:
from pyspark import SparkContext
from pyspark import SparkConf
sc =SparkContext()

# 利用Spark 分析台灣2015 PM2.5資料集

<a id="load_data"></a>
## 上傳台灣2015一整年空氣監測資料 

### 步驟1: 將2015空氣監控資料，上傳至colab

In [ ]:
!wget -O pm25.csv "https://www.dropbox.com/s/zkn3ba7pitv83el/pm2.5Taiwan.csv?dl=0"

In [ ]:
!ls

In [ ]:
weather = sc.textFile("./pm25.csv")

### 步驟2: 試試看是否成功上傳  (使用count( ), first( ), collect( ), take( ) )

In [ ]:
weather.count()
weather.first()

In [ ]:
weather.take(5)

# 練習1: 讓我們求取2015年，大里每小時的平均pm25數值。
## 注意事項：
1. 資料分割：原始資料每一行為一個觀測值，我們必須將資料進行分割，才能逐一計算與進行操作。
2. 資料清洗：在氣象局的原始資料裡，有些數值由於當初偵測時有異常，所以會加註特別符號如\*\#等特殊符號，這些數值我們必須先經過前處理，我們才能進行算術運算。
3. 資料選擇：將大里資料挑選出來
4. 產生key-value，也就是(小時,pm25值)
5. 利用flatMap(), reduceByKey(), groupByKey()，將不同日期但相同時間的pm25值收集起來。
6. 計算平均值, 標準差, 最大最小值。


### 步驟一：資料分割 (使用map () 與 split( ))

In [ ]:
weatherParse = weather.map(lambda line : line.split(","))
print(weatherParse.first())

### 步驟二：將大里站資料從全部資料集中挑選出來 (filter)

In [ ]:
wea_dali = weatherParse.filter(lambda x: x[1] == '大里' and x[2]== "PM2.5")
print(wea_dali.take(50))

### 步驟三：資料清洗 (使用 map(), str.strip())
#### 在氣象局的原始資料裡，有些數值由於當初偵測時有異常，所以會加註特別符號如\*\#等特殊符號，或者沒有取到數值為一空值，這些數值我們必須先經過前處理，我們才能進行算術運算。
1. ```2015/01/29 大里 PM2.5 14 14 12 7 1 0 4 6 12 16 15  52x 38x 30 29 27 29 24 24 21 19 19 23```
2. ```2015/01/16 大里 PM2.5 16 15 17 16 16 13 5 10 14 30 30 25 -4# 22 23 30 33 40 43 45 37 34 38 43``` 

In [ ]:
def strip_symbol(x):
    for i in range(len(x)):
        x[i] = x[i].strip("-*#x") # remove non-digits
        if x[i]=="": x[i]="0"
    return x

wea_dali = wea_dali.map(strip_symbol)

### 步驟四：key value pair的產生 (重要的操作概念)
*將每小時資料轉成(小時,pm數值)，以求取每小時的平均值。

例如：
    2015/01/01 大里 PM2.5 53 55 58 53 43 36 35 42 55 64 65 59 52 44 47 41 43 40 42 35 28 20 18 16
    --> [(0, 53) (1, 55) (2, 58) (3, 53) (4, 43) ... (21, 20) (22, 18) (23, 16)]


In [ ]:
print(wea_dali.first())

In [ ]:
def hourKeyGen(x):
    hourkeypair = []
    x=x[3:]
    for i, value in enumerate(x):
      print(i, value)
      hourkeypair.append((i, float(value)))
    return hourkeypair

wea_dali_byHourkey = wea_dali.map(hourKeyGen)

wea_dali_byHourkey.first()

### 步驟五： 利用flatMap(), reduceByKey(), groupByKey()，將不同日期但相同時間的pm25值收集起來。(使用flatMap)

In [ ]:
byHourkey = wea_dali.flatMap(hourKeyGen)
byHourkey.reduceByKey(lambda x,y: x+y).take(10)

### 步驟六： 計算大里區每個小時區間中，平均之pm25數值 (使用reduceByKey)

In [ ]:
avg_pm25_hour = byHourkey.reduceByKey(lambda x,y: x+y)
avg_pm25_hour.map(lambda x:(x[0],x[1]/365)).collect()

### 步驟七： 根據pm25平均濃度，進行排序。使用top( )

In [ ]:
avg_pm25_hour = byHourkey.reduceByKey(lambda x,y: x+y)
avg_pm25_hour.map(lambda x:(x[0],x[1]/365.0)).map(lambda x: (x[1],x[0])).top(24)

### 步驟八： 計算每個時間點的統計值，例如最大值、最小值、平均值、標準差(使用 groupByKey()與mapValues())

In [ ]:
import numpy as np

hour_stat_list = byHourkey.groupByKey().mapValues(list).collect()

for i in sorted(hour_stat_list):
    print (i[0],max(i[1]),min(i[1]),np.mean(i[1]),np.var(i[1]))

# 練習2: 請求取2015年，全國pm2.5最高的前十個工作站測點以及其日期。

In [ ]:
### weatherParse 為原始資料集
### 資料清洗
def strip_symbol(x):
    for i in range(len(x)):
        #x[i] = x[i].strip("-*#x") # remove non-digits
        if "#" in x[i]:x[i]="0"
        if "*" in x[i]:x[i]="0"
        if "x" in x[i]:x[i]="0"
        if "-" in x[i]:x[i]="0"
        if x[i]=="": x[i]="0"
    return x

def date_location_time (x):
    date_loc_time_measure = []
    date = x[0]
    loc = x[1]
    x = x[3:]
    for hour, i in enumerate(x):
      date_loc_time_string = date+" at "+ loc +" : "+str(hour)+ u'點'
      date_loc_time_measure.append((date_loc_time_string, int(i)))
    
    return date_loc_time_measure

In [ ]:
weather = sc.textFile("./pm25.csv")
weatherParse = weather.map(lambda line : line.split(","))

top10pm25 = weatherParse.filter(lambda x: x[2]=="PM2.5")\
            .map(strip_symbol)\
            .flatMap(date_location_time)\
            .map(lambda x: (x[1],x[0]))\
            .top(10)          

for i in top10pm25:
    print(i)

In [ ]:
precTop10=[]
stationsTop10=[]
for result in top10pm25:
    precTop10.append(result[0])
    stationsTop10.append(result[1])

Plot your results.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

N = 10
index = np.arange(N)  
bar_width = 0.5

plt.bar(index, precTop10, bar_width, color='b')
plt.xlabel('Stations')
plt.ylabel('pm25')
plt.title('10 stations with the highest average pm25')
plt.xticks(index + bar_width, stationsTop10, rotation=90)
plt.show()

# 練習3: 請算算看2015全國哪個測站，紫爆天數最多？
### 假設當日平均值大於60，則算該日該地區紫爆


In [ ]:

### 資料清洗
def strip_symbol(x):
    for i in range(len(x)):
        #x[i] = x[i].strip("-*#x") # remove non-digits
        if "#" in x[i]:x[i]="0"
        if "*" in x[i]:x[i]="0"
        if "x" in x[i]:x[i]="0"
        if "-" in x[i]:x[i]="0"
        if x[i]=="": x[i]="0"
    return x

def computeAverage(x):
    location = x[1]
    x = x[3:]
    y = [float(i) for i in x]
    return (location, sum(y)/len(y))

Worst = weatherParse\
        .map(strip_symbol)\
        .filter(lambda x: x[2]=="PM2.5")\
        .map(computeAverage)\
        .filter(lambda x:x[1]>60)\
        .map(lambda x: (x[0],1))\
        .reduceByKey(lambda x,y:x+y)\
        .sortBy(lambda x:x[1], ascending=False).collect()


for loc in Worst:
  print(loc)


